In [1]:
from bs4 import BeautifulSoup
import urllib
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 

In [2]:
#class for rendering js pages for use with BS
class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  
  
    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit() 

In [3]:
#list of all mathces in list links
def season_links(year):
    resultsPage = urllib.urlopen("http://www.fourfourtwo.com/statszone/results/8-" + year)
    resultsPageSoup = BeautifulSoup(resultsPage)
    scores = resultsPageSoup.findAll('td', {'class': 'score'})
    homeTeams = resultsPageSoup.findAll('td', {'class': 'home-team'})
    awayTeams = resultsPageSoup.findAll('td', {'class': 'away-team'})
    gamesStatsLinks = ['http://www.fourfourtwo.com' + 
                       l.a['href'] for l in resultsPageSoup.findAll('td', {'class': 'link-to-match'})]
    return(gamesStatsLinks)

In [4]:
links = season_links('2015')

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
def match_stats(link):
    resultsPage = urllib.urlopen(link)
    resultsPageSoup = BeautifulSoup(resultsPage)

    #overall statistics
    match_stats = {}
    match_stats['possession'] = resultsPageSoup.findAll('div', {'id':'summary_possessions'})[0].findAll('svg', {'class': 'doughnut_chart'})[0].text

    match_stats['home_pass_cmplt'] = resultsPageSoup.findAll('div', {'id':'summary_passes'})[0].findAll('text', {'fill': "#FCD800"})[0].next
    match_stats['away_pass_cmplt'] = resultsPageSoup.findAll('div', {'id':'summary_passes'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next

    match_stats['home_crnr'] = resultsPageSoup.findAll('div', {'id':'summary_corners'})[0].findAll('text', {'fill': "#FCD800"})[0].next
    match_stats['away_crnr'] = resultsPageSoup.findAll('div', {'id':'summary_corners'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next

    match_stats['home_attck_pass'] = resultsPageSoup.findAll('div', {'id':'summary_attacking'})[0].findAll('text', {'fill': "#FCD800"})[0].next
    away_attck_pass = resultsPageSoup.findAll('div', {'id':'summary_attacking'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next

    match_stats['home_shots'] = resultsPageSoup.findAll('div', {'id':'summary_shots'})[0].findAll('text', {'fill': "#FCD800"})[0].next
    match_stats['away_shots'] = resultsPageSoup.findAll('div', {'id':'summary_shots'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next

    match_stats['home_fouls'] = resultsPageSoup.findAll('div', {'id':'summary_fouls'})[0].findAll('text', {'fill': "#FCD800"})[0].next
    match_stats['away_fouls'] = resultsPageSoup.findAll('div', {'id':'summary_fouls'})[0].findAll('text', {'fill': "#E6E6E6"})[0].next

    match_stats['date_location'] = resultsPageSoup.findAll('div', {'class': 'teams'})[0].next.strip()
    return(match_stats)


In [6]:
stats_test = match_stats(links[1])

In [7]:
stats_test

{'away_crnr': u'4',
 'away_fouls': u'6',
 'away_pass_cmplt': u'334',
 'away_shots': u'5',
 'date_location': u'Emirates Stadium London, Sunday, May 15, 2016 - 15:00',
 'home_attck_pass': u'283',
 'home_crnr': u'5',
 'home_fouls': u'12',
 'home_pass_cmplt': u'537',
 'home_shots': u'16',
 'possession': u'59.6%40.4%'}

In [6]:
def player_links(game_link):
    url = game_link + '/player-stats#tabs-wrapper-anchor'
    r = Render(url)
    result = r.frame.toHtml()
    formatted_result = str(result.toAscii())
    soup = BeautifulSoup(formatted_result, 'html5')
    player_urls = ['http://www.fourfourtwo.com' + a['href'] 
               for a in soup.findAll('div', {'id':'lineups'})[0].findAll('a')]
    player_names = [a.contents
               for a in soup.findAll('div', {'id':'lineups'})[0].findAll('a')]
    
    players = {}
    for i in range(len(player_names)):
        players[player_names[i][0]] = {"link":player_urls[i] }
    
    return(players)


In [7]:
players = player_links(links[1])

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [5]:
def player_stats(url):
    #
    playerPage = Render(url)
    playerPageRendered = playerPage.frame.toHtml()
    playerPageFormatted = str(playerPageRendered.toAscii())
    playerSoup = BeautifulSoup(playerPageFormatted, 'html5')
    
    match_stats = {}
    #passes
    match_stats['pass_success'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smallblue)'}))
    match_stats['pass_fail'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smallred)'}))
    match_stats['chances'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smalldeepskyblue)'}))
    match_stats['assists'] = len(playerSoup.findAll('line', {'marker-end': 'url(#smallyellow)'}))

    #shots
    match_stats['on_target'] = len(playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'}))
    match_stats['off_target'] = len(playerSoup.findAll('line', {'marker-end': 'url(#bigredend)'}))
    match_stats['on_target'] = len(playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'}))
    match_stats['goals'] = len(playerSoup.findAll('line', {'marker-start': 'url(#bigyellowend)'}))

    #takeons
    match_stats['takeon_success'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'}))
    match_stats['takeon_fail'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/fail.png'}))

    #defensive
    match_stats['fail_tkl'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_tackle.png'}))
    match_stats['success_tkl'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png'}))
    match_stats['success_clearence'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_clearence.png'}))
    match_stats['failed_clearence'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_clearence.png'}))
    match_stats['interception'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/interceptions.png'}))
    match_stats['ball_recovery'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/defensive-ball-clearence.png'}))
    match_stats['blocks'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks.png'}))
    match_stats['blocks_cross'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks-cross.png'}))

    #aerial
    match_stats['aerial_won'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/won.png'}))
    match_stats['aerial_lost'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/lost.png'}))

    #fouls
    match_stats['foul_cmt'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/commited.png'}))
    match_stats['foul_sfr'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/suffered.png'}))

    #errors
    match_stats['error_goal'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-goal.png'}))
    match_stats['error_shot'] = len(playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-shot.png'}))
    
    return(match_stats)
    
    
    

In [ ]:
url = players['Gueye']['link']

In [6]:
stats = player_stats('http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80801/OVERALL_02#tabs-wrapper-anchor')

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [7]:
stats

{'aerial_lost': 0,
 'aerial_won': 0,
 'assists': 0,
 'ball_recovery': 0,
 'blocks': 0,
 'blocks_cross': 0,
 'chances': 1,
 'error_goal': 0,
 'error_shot': 0,
 'fail_tkl': 0,
 'failed_clearence': 0,
 'foul_cmt': 2,
 'foul_sfr': 1,
 'goals': 0,
 'interception': 1,
 'off_target': 0,
 'on_target': 0,
 'pass_fail': 7,
 'pass_success': 53,
 'success_clearence': 0,
 'success_tkl': 2,
 'takeon_fail': 2,
 'takeon_success': 2}

In [10]:
len(stats['aerial_lost'])

0

In [6]:
playerPage = Render('http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/80801/OVERALL_02#tabs-wrapper-anchor')
playerPageRendered = playerPage.frame.toHtml()
playerPageFormatted = str(playerPageRendered.toAscii())
playerSoup = BeautifulSoup(playerPageFormatted, 'html5')

/home/lucas/anaconda2/envs/football/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/lucas/anaconda2/envs/football/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [ ]:

#
playerPage = Render(url)
playerPageRendered = playerPage.frame.toHtml()
playerPageFormatted = str(playerPageRendered.toAscii())
playerSoup = BeautifulSoup(playerPageFormatted, 'html5')

match_stats = {}
#passes
match_stats['pass_success'] = playerSoup.findAll('line', {'marker-end': 'url(#smallblue)'})
match_stats['pass_fail'] = playerSoup.findAll('line', {'marker-end': 'url(#smallred)'})
match_stats['chances'] = playerSoup.findAll('line', {'marker-end': 'url(#smalldeepskyblue)'})
match_stats['assists'] = playerSoup.findAll('line', {'marker-end': 'url(#smallyellow)'})

#shots
match_stats['on_target'] = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
match_stats['off_target'] = playerSoup.findAll('line', {'marker-end': 'url(#bigredend)'})
match_stats['on_target'] = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
match_stats['goals'] = playerSoup.findAll('line', {'marker-start': 'url(#bigyellowend)'})

#takeons
match_stats['takeon_success'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'})
match_stats['takeon_fail'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/fail.png'})

#defensive
match_stats['fail_tkl'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_tackle.png'})
match_stats['success_tkl'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png'})
match_stats['success_clearence'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_clearence.png'})
match_stats['failed_clearence'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_clearence.png'})
match_stats['interception'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/interceptions.png'})
match_stats['ball_recovery'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/defensive-ball-clearence.png'})
match_stats['blocks'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks.png'})
match_stats['blocks_cross'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks-cross.png'})

#aerial
match_stats['aerial_won'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/won.png'})
match_stats['aerial_lost'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/lost.png'})

#fouls
match_stats['foul_cmt'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/commited.png'})
match_stats['foul_sfr'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/suffered.png'})

#errors
match_stats['error_goal'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-goal.png'})
match_stats['error_shot'] = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-shot.png'})





In [7]:
players.keys()[1]

u'Gueye'

In [ ]:
stats = player_stats(players['Gueye']['link'])

In [3]:
url = 'http://www.fourfourtwo.com/statszone/8-2015/matches/803532/player-stats/37265/OVERALL_02#tabs-wrapper-anchor'
playerPage = Render(url)
playerPageRendered = playerPage.frame.toHtml()
playerPageFormatted = str(playerPageRendered.toAscii())
playerSoup = BeautifulSoup(playerPageFormatted, 'html5')


/Users/lucaslafone/anaconda/envs/MLlib/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [5]:
soup.findAll('line', {'marker-start': 'url(#bigyellowend)'})

[<line class="pitch-object timer-1-4" marker-end="url(#bigyellow)" marker-start="url(#bigyellowend)" style="stroke:yellow;stroke-width:3" x1="621.16" x2="692" y1="258.095" y2="246.139"></line>,
 <line class="pitch-object timer-2-77" marker-end="url(#bigyellow)" marker-start="url(#bigyellowend)" style="stroke:yellow;stroke-width:3" x1="658.512" x2="692" y1="230.34" y2="264.073"></line>,
 <line class="pitch-object timer-2-79" marker-end="url(#bigyellow)" marker-start="url(#bigyellowend)" style="stroke:yellow;stroke-width:3" x1="643.7" x2="692" y1="234.183" y2="255.96"></line>]

In [4]:
soup.findAll('image', {'xlink:href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'})

[]

In [15]:
len(chances)

4

In [5]:

#passes
pass_success = playerSoup.findAll('line', {'marker-end': 'url(#smallblue)'})
pass_fail = playerSoup.findAll('line', {'marker-end': 'url(#smallred)'})
chances = playerSoup.findAll('line', {'marker-end': 'url(#smalldeepskyblue)'})
assists = playerSoup.findAll('line', {'marker-end': 'url(#smallyellow)'})

#shots
on_target = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
off_target = playerSoup.findAll('line', {'marker-end': 'url(#bigredend)'})
on_target = playerSoup.findAll('line', {'marker-end': 'url(#bigblueend)'})
goals = playerSoup.findAll('line', {'marker-start': 'url(#bigyellowend)'})

#takeons
takeon_success = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/success.png'})
takeon_fail = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/fail.png'})

#defensive
fail_tkl = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_tackle.png'})
success_tkl = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_tackle.png'})
success_clearence = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/successful_clearence.png'})
failed_clearence = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/failed_clearence.png'})
interception = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/interceptions.png'})
ball_recovery = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/defensive-ball-clearence.png'})
blocks = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks.png'})
blocks_cross = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/blocks-cross.png'})

#aerial
aerial_won = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/won.png'})
aerial_lost = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/lost.png'})

#fouls
foul_cmt = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/commited.png'})
foul_sfr = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/suffered.png'})

#errors
error_goal = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-goal.png'})
error_shot = playerSoup.findAll('image', {'href': '/sites/fourfourtwo.com/modules/custom/statzone/files/icons/error-leading-shot.png'})


In [7]:
len(goals)
len(pass_success)

42

[<image height="529" width="740" x="0" xlink:href="/sites/fourfourtwo.com/modules/custom/statzone/files/statszone_football_pitch.png" y="0"></image>]